In [2]:
" Node2Vec model using torch geometric with Cora"
import torch
from torch_geometric.datasets import Planetoid # The citation network datasets “Cora”, “CiteSeer” and “PubMed” 
from torch_geometric.nn import Node2Vec # Import Node2Vec Model
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import os

In [3]:
" **************** IMPORT DATA ********************"
path = os.getcwd() # Directory to download dataset
dataset_name = "PubMed"
dataset = Planetoid(path, dataset_name) # Download the dataset: “Cora”, “CiteSeer” and “PubMed” 
data = dataset[0] # Tensor representation of the Cora-Planetoid data
print(f'{dataset_name}: ', data)

device = 'cpu'


PubMed:  Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [4]:
" **************** CONSTRUCT THE MODEL  ********************"
Node2Vec_model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
                     context_size=10, walks_per_node=10,
                     num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = Node2Vec_model.loader(batch_size=100, shuffle=True, num_workers=4) # For batch training
optimizer = torch.optim.SparseAdam(list(Node2Vec_model.parameters()), #List of parameters
                                   lr = 0.01 # Learning Rate
                                   )


In [5]:
" **************** TRAIN FUNCTION ********************"
def train():
    Node2Vec_model.train() # Set training as true for the model
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad() # reset of gradient of all variables
        loss = Node2Vec_model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss =+ loss.item()
    return total_loss / len(loader)


In [ ]:
" **************** GET EMBEDDING  ********************"
if __name__ == "__main__":
    for epoch in range(1, 101):
        loss = train()
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

In [ ]:
" **************** PLOT 2D OF EMBEDDED REP   ********************"
@torch.no_grad() # Deactivate autograd functionality
def plot_point(colors):
    Node2Vec_model.eval() # Evaluate the model based on the trained parameters
    z = Node2Vec_model(torch.arange(data.num_nodes, device=device)) # Embedding rep
    z = TSNE(n_components=2).fit_transform(z.cpu().numpy())
    y = data.y.cpu().numpy()
    plt.figure()
    for i in range(dataset.num_classes):
        plt.scatter(z[y==i,0],z[y==i,1],s=20,color=colors[i])
    plt.axis('off')
    plt.show()
colors = [
        '#ffc0cb', '#bada55', '#008080', '#420420', '#7fe5f0', '#065535',
        '#ffd700'
    ]
plot_point(colors)


In [ ]:
" **************** NODE CLASSIFICATION ********************"
def test():
    Node2Vec_model.eval() # Evaluate the model based on the trained parameters
    z = Node2Vec_model() # Evaluate the model based on the trained parameters
    acc = Node2Vec_model.test(z[data.train_mask] ,data.y[data.train_mask],
                              z[data.test_mask],data.y[data.test_mask],
                              max_iter=150)
    return acc

print('Accuracy:', test())

## Other example

In [19]:
from torch_geometric.nn import Node2Vec
import os.path as osp
import torch
from torch_geometric.datasets import Planetoid
from tqdm.notebook import tqdm

dataset = 'Cora'
path = osp.join('.', 'data', dataset)
dataset = Planetoid(path, dataset)  # dowload or load the Cora dataset
data = dataset[0]
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # check if cuda is available to send the model and tensors to the GPU
model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
                 context_size=10, walks_per_node=10,
                 num_negative_samples=1, p=1, q=1, sparse=True).to(device)


loader = model.loader(batch_size=128, shuffle=True, num_workers=0)  # data loader to speed the train 
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)  # initzialize the optimizer 


def train():
    model.train()  # put model in train model
    total_loss = 0
    for pos_rw, neg_rw in tqdm(loader):
        optimizer.zero_grad()  # set the gradients to 0
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))  # compute the loss for the batch
        loss.backward()
        optimizer.step()  # optimize the parameters
        total_loss += loss.item()
    return total_loss / len(loader)


for epoch in range(1, 100):
    loss = train()
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')

all_vectors = ""
for tensor in model(torch.arange(data.num_nodes, device=device)):
    s = "\t".join([str(value) for value in tensor.detach().cpu().numpy()])
    all_vectors += s + "\n"
# save the vectors
with open("vectors.txt", "w") as f:
    f.write(all_vectors)
# save the labels
with open("labels.txt", "w") as f:
    f.write("\n".join([str(label) for label in data.y.numpy()]))

Processing...
Done!


  0%|          | 0/22 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 10248, 26392, 3768, 12020) exited unexpectedly